Add necessary imports here

In [1]:
import nltk, string, os, json

from nltk import RegexpParser
from nltk.tree import Tree

from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Add necessary dictionaries/lists and parsing methods here

In [0]:
def get_article(filename):
    with open(filename, "r") as f:
        s = [] # steps - list of tuples (method_number, step text)
        d = {} # descriptions - Description dictionary step text -> description
        m = [] # methods
        data_json = json.load(f)
        title = data_json['title']
        if title != None:
        # Get the steps
            if len(data_json['method/part']) > 0:
                methods = data_json['method/part']
                for method in methods:
                    all_steps = [(method['name'], step['headline']) for step in method['steps']]
                    s += all_steps
                    m += method['name']
                    all_descs = dict(zip([step['headline'] for step in method['steps']], [step['description'] for step in method['steps']]))
                    d.update(all_descs)      
            else:
              print(filename)
               # s = [(None, step[0]) for step in data_json['steps']]
               # d = dict(zip([step[0] for step in data_json['steps']], [step[1] for step in data_json['steps']]))
        else:
            print('Article has no title!')
        return s, d, m, title

In [36]:
nltk.download() # download averaged_perceptron_tagger, wordnet
from nltk.corpus import wordnet

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


Add NLTK POS-tagging and tokenizer here

In [0]:
# Source: https://stackoverflow.com/questions/29473169/approach-for-identifying-whether-a-sentence-includes-an-imperative-within-it?fbclid=IwAR1kGWHvo0M1sSMMXl4Rv4PJ-dhRchEABc83JQpAqotQnFqY3uUBD0jAMAU

def is_imperative(tagged_sent):
    # if the sentence is not a question...
    if tagged_sent[-1][0] != "?":
         #catches simple imperatives, e.g. "Open the pod bay doors, HAL!"
        
        if tagged_sent[0][1] == "VB" or tagged_sent[0][1] == "MD":
            return True
        
        # catches imperative sentences starting with words like 'please', 'you',...
        # E.g. "Dave, stop.", "Just take a stress pill and think things over."
        else:
          # check if the first word has a verb synonym
          synonyms = [syns.name() for syns in wordnet.synsets(tagged_sent[0][0])]
          # split the synonyms array into two: more related synonyms should appear earlier
          # Follow a heuristic such that if a verb synonym doesn't appear in the first half
          # then this word is likely more often used as a noun
          synonyms = synonyms[0:len(synonyms)//2]
          for synonym in synonyms:
            # if there is a verb written the same way as the first token, return true
            splitted = synonym.split('.')
            if splitted[1] == 'v':
              if splitted[0] == tagged_sent[0][0].lower():
                return True
          #chunk = get_chunks(tagged_sent)
          # check if the first chunk of the sentence is a VB-Phrase
          #if type(chunk[0]) is Tree and chunk[0].label() == "VB-Phrase":
              #return True
    # For the sake of our game, consider the questions as non-imperative  
    return False

# chunks the sentence into grammatical phrases based on its POS-tags
def get_chunks(tagged_sent):
    chunkgram = r"""VB-Phrase: {<DT><,>*<VB>}
                    VB-Phrase: {<RB><VB>}
                    VB-Phrase: {<UH><,>*<VB>}
                    VB-Phrase: {<UH><,><VBP>}
                    VB-Phrase: {<PRP><VB>}
                    VB-Phrase: {<NN.?>+<,>*<VB>}
                    Q-Tag: {<,><MD><RB>*<PRP><.>*}"""
    chunkparser = RegexpParser(chunkgram)
    return chunkparser.parse(tagged_sent)

# Tokenizer that I implemented from CIS421
def tokenize(text):
    tokens = []
    words = text.split(' ')
    for word in words:
        if len(word) == 0:
            continue
        w = []
        for c in word:
            if c in string.punctuation:
              if c == '\'':
                w.append(c)
              else:
                  if len(w) != 0:
                      tokens.append(''.join(w))
                      w = []
                  tokens.append(c)
            else:
                if c not in string.whitespace:
                    w.append(c)
        if len(w) != 0:
            tokens.append(''.join(w))
    return tokens

def imperative_check(text):
  # First tokenize the step headline
  tokens = tokenize(text)
  # Get the POS-tags
  tags = nltk.pos_tag(tokens)
  # Check if imperative
  imperative = is_imperative(tags)
  
  return imperative

Add similarity check functions here

In [6]:
!pip3 install pymagnitude
!wget http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude

from pymagnitude import *
vectors = Magnitude("glove.6B.300d.magnitude")

--2020-04-18 02:12:02--  http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude
Resolving magnitude.plasticity.ai (magnitude.plasticity.ai)... 52.216.29.115
Connecting to magnitude.plasticity.ai (magnitude.plasticity.ai)|52.216.29.115|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1384890368 (1.3G) [binary/octet-stream]
Saving to: ‘glove.6B.300d.magnitude.1’

glove.6B.300d.magni 100%[===================>]   1.29G  16.5MB/s    in 81s     

2020-04-18 02:13:24 (16.2 MB/s) - ‘glove.6B.300d.magnitude.1’ saved [1384890368/1384890368]



In [0]:
#get vector for the given sentence
def construct_sentence_vector(command, vectors):
  sentence_vector = np.zeros(shape=(vectors.dim,))
  for word in command.split():
    word_vector = vectors.query(word)
    sentence_vector += word_vector
  sentence_vector = sentence_vector / len(command.split())
  return sentence_vector

#compare items in the list to the article_field. for instance, if you want to find the most similar titles
#article_field should be the title, and list should be a list of titles you want to find the most similar out of
def find_most_similar(article_field, list, vectors):
  vecs = {}
  max_sim = 0
  most_sim = list[0]
  vector_1 = construct_sentence_vector(article_field, vectors)
  for obj in list:
    vecs[obj] = construct_sentence_vector(obj, vectors)
  for obj in vecs:
    sim = vectors.similarity(vecs[obj], vector_1)
    if sim > max_sim:
      most_sim = obj
      max_sim = sim
  return most_sim

#get the top k most similar
def top_k_similar(article_field, list, vectors, k):
  top = []
  for i in range(k):
    most_sim = find_most_similar(article_field, list, vectors)
    top.append(most_sim)
    list.remove(most_sim)
  return top


Add object identifier & 3rd person sentence formation here

In [0]:
# refered from https://stackoverflow.com/questions/41051125/turning-a-sentence-from-first-to-second-person

forms = {"am" : "are", "are" : "am", 'i' : 'you', 'my' : 'your', 'me' : 'you', 'mine' : 'yours', 'you' : 'I', 
'your' : 'my', 'yours' : 'mine'}

def translate(text):
  # print(text)
  result = text
  for word in text.split():
    # print(word)
    if word.lower() in forms: 
      result = result.replace(word, forms[word.lower()])
      # print(result)
  # print(result)
  return result

def changePerson(text):
  result = ""
  tags = nltk.pos_tag(tokenize(text))
  isImp = is_imperative(tags)

  if isImp:
    if text[0] == ' ' or text[0] == '\n':
      result = "You " + text[1].lower() + text[2:]
    else:
      result = "You " + text[0].lower() + text[1:]
  else:
    result = translate(text)
  return result

def identifyObjects(text):
  tags = nltk.pos_tag(tokenize(text))
  objects = []
  for t in tags:
    if t[1]=="NN":
      objects.append(t[0])
  return objects

Once imports and functions are done, upload a JSON file, get its name and parse it

In [97]:
uploaded = files.upload()

Saving 3029.json to 3029 (2).json


In [0]:
file_name = list(uploaded.keys())[0]

In [99]:
def trim_beginning_punctuation(sentence):
  to_return = ''
  letter_seen = False
  for c in sentence:
    if c in string.punctuation or c in string.whitespace:
      if letter_seen:
        to_return += c
    else:
      if c in string.ascii_letters:
        if not letter_seen:
          letter_seen = True
        to_return += c
  return to_return

def milestone_1_results(filename):
  steps, descriptions, method_names, title = get_article(filename)
  # print the title of the article
  print('Title: ' + title)
  # Once the necessary fields are filled, find imperatives
  imperatives = []

  for step in steps:
    # First split the headline into sentences
    headline_sentences = step[1].split('.')
    for headline in headline_sentences:
      trimmed_headline = trim_beginning_punctuation(headline)
      trimmed_headline = trimmed_headline.replace('\n', '')
      if len(trimmed_headline) == 0:
          continue
      if imperative_check(trimmed_headline):
        imperatives.append(trimmed_headline)
        break # No two imperatives from the same headline are added.
    description = descriptions[step[1]]
    # Split the description paragraph into sentences
    sentences = description.split('.')
    for sentence in sentences:
      trimmed_sentence = trim_beginning_punctuation(sentence)
      trimmed_sentence = trimmed_sentence.replace('\n', '')
      if len(trimmed_sentence) == 0:
        continue
      if imperative_check(trimmed_sentence):
        imperatives.append(trimmed_sentence)
        break # No two imperatives from the same description are added. !!! REMOVE THIS IF you want multiple imperatives

    
  # Once imperatives are found, print them and turn them into 3rd person
  print('\nList of Imperatives:\n')
  for imp in imperatives:
    print(imp)
    
  third_person = [changePerson(imperative) for imperative in imperatives]

  # Print the 3rd person sentences in order
  print('\nImperatives Turned Into 3rd Person:\n')
  for third in third_person:
    print(third)
  
  # # Find 5 similar articles to this article given the title
  # k = 5

  # # Make sure that the folder new_wikihow_1000 is in the drive - that's where we have all the data
  # file_path = '/content/drive/My Drive/Colab Notebooks/new_wikihow_1000'
  # all_files = os.listdir(file_path)
  # files_and_titles = {} # title -> filename dictionary
  # idx = 0
  # for data in all_files:
  #   if idx >= 100: # for now the number of files to read is limited to 100
  #     break
  #   with open(file_path + '/' + data, "r") as f:
  #     data_json = json.load(f)
  #     t = data_json['title']
  #     # Skip articles with no title
  #     if t == None:
  #       continue
  #     files_and_titles[t] = data
  #     idx += 1
  
  # # Once we have file titles, pass them as a list to top_k_similar()
  # k_similar = top_k_similar(title, list(files_and_titles.keys()), vectors, k)
  
  # # Print the top k similar article titles and the corresponding json file name
  # print('\nTop ' + str(k) + ' Similar Article Titles:\n')
  # for similar in k_similar:
  #   print('Title: ' + similar + ', Filename: ' + files_and_titles[similar])

milestone_1_results(file_name)

Title: How to Get Gigs for Your Band

List of Imperatives:

Reach out to people in your network
Let your friends, family, coworkers, classmates, and anyone else you're acquainted with know that you're in a band and you're looking for gigs
Give the promoter a call and tell them your band is interested in doing a show at their venue
Call local promoters and ask if you can stop by for a few minutes to drop off a sample of your music
Make recordings of your music to play to venues
Try to have at least - tracks recorded before you start pitching
Follow similar bands on social media and keep track of where they're performing
Offer to perform on a weeknight
Tell venues you're available for any open slots, including weeknights
Take high pay\/low fan gigs to fund your band
Take the money you make from those gigs and use it to pay for the expenses of playing a low pay\/high fan gig where you'll get more exposure
Look for competitions that offer prizes like a slot in a local music festival lineup